In [33]:
import sys
import os
from metient.metient import *
from metient.util import data_extraction_util as dutil

REPO_DIR = os.path.join(os.getcwd(), "../../")
TRACERX_DATA_DIR = os.path.join(REPO_DIR, 'data/tracerx_nsclc/')
INPUT_DIR = os.path.join(TRACERX_DATA_DIR, 'tracerx_provided_patient_data', )
OUTPUT_DIR = os.path.join(TRACERX_DATA_DIR, "metient_outputs", "test")

def run_conipher_patient(patient, weights):
    space = "x"*44
    tsv_fn = os.path.join(INPUT_DIR, f"{patient}_SNVs.tsv")
    print(tsv_fn)
    print(f"{space} PATIENT {patient} {space}")
    tree_fn = os.path.join(INPUT_DIR, f"{patient}_tree.txt")
    tree = dutil.get_adjacency_matrix_from_txt_edge_list(tree_fn)
    print_config = PrintConfig(visualize=False, verbose=False, k_best_trees=30)
    evaluate_label_clone_tree(tree, tsv_fn, weights, print_config, OUTPUT_DIR, patient, O=None,
                              solve_polytomies=True, batch_size=4096)
#     calibrate_label_clone_tree([tree], [tsv_fn], print_config, OUTPUT_DIR, [patient], Os=None,
#                                solve_polytomies=True, batch_size=-1)
    

In [35]:
# %%prun
weights = Weights(mig=1, comig=50, seed_site=100, reg=2, gen_dist=0.1, organotrop=0.0)

run_conipher_patient("CRUK0698", weights)

  0%|          | 1/450 [00:00<00:50,  8.85it/s]

/lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/tracerx_analysis/../../data/tracerx_nsclc/tracerx_provided_patient_data/CRUK0698_SNVs.tsv
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx PATIENT CRUK0698 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
/lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/tracerx_analysis/../../data/tracerx_nsclc/tracerx_provided_patient_data/CRUK0698_SNVs.tsv
ordered_sites ['LN_SU_LN1', 'primary_SU_T1', 'LN_SU_FLN1']


100%|██████████| 450/450 [00:53<00:00, 18.57it/s]

pareto_metrics {(6, 2, 1), (4, 2, 2)}
# final solutions: 3


100%|██████████| 450/450 [01:10<00:00, 18.57it/s]

In [3]:

tracerx_reported_counts = {
    "Brain":6, "Chest wall":1, "Bronchus":4,"Pleura":2,"Rib":2,
    "Liver":6,"Flank":1,"Femur":1,"Neck LN":2,"Primary LN":148,
    "Recurrence LN":20,"Scapula":1,"Axillary LN":1,"Recurrence lung":13,
    "Primary surgery satellite lesion lung":3, "Adrenal gland":1,
    "Subphrenic mass":1, "Skin":3, "Paravertebral":1, "Peritoneum":1,
    "Lung":0
}

total = sum([tracerx_reported_counts[key] for key in tracerx_reported_counts])
tracerx_reported_freqs = {key:float(tracerx_reported_counts[key]/total) for key in tracerx_reported_counts}
print(tracerx_reported_freqs)
def get_organotropism_matrix_from_tracerx(unique_sites, tracerx_map):
    mapped_sites = [tracerx_map[site] for site in unique_sites]
    organotrop_arr = np.zeros(len(mapped_sites))
    for i, metastatic_site in enumerate(mapped_sites):
        organotrop_arr[i] = tracerx_reported_freqs[metastatic_site]
    return torch.tensor(organotrop_arr, dtype = torch.float32)


{'Brain': 0.027522935779816515, 'Chest wall': 0.0045871559633027525, 'Bronchus': 0.01834862385321101, 'Pleura': 0.009174311926605505, 'Rib': 0.009174311926605505, 'Liver': 0.027522935779816515, 'Flank': 0.0045871559633027525, 'Femur': 0.0045871559633027525, 'Neck LN': 0.009174311926605505, 'Primary LN': 0.6788990825688074, 'Recurrence LN': 0.09174311926605505, 'Scapula': 0.0045871559633027525, 'Axillary LN': 0.0045871559633027525, 'Recurrence lung': 0.05963302752293578, 'Primary surgery satellite lesion lung': 0.013761467889908258, 'Adrenal gland': 0.0045871559633027525, 'Subphrenic mass': 0.0045871559633027525, 'Skin': 0.013761467889908258, 'Paravertebral': 0.0045871559633027525, 'Peritoneum': 0.0045871559633027525, 'Lung': 0.0}


100%|██████████| 450/450 [01:00<00:00, 27.81it/s]

In [4]:
import pandas as pd
no_cna_dir = os.path.join(TRACERX_DATA_DIR, "CRUK0762_no_cna_testing")
dutil.pool_input_tsv(os.path.join(no_cna_dir, "CRUK0762_SNVs.tsv"), no_cna_dir, f"tmp_CRUK0762")
df = pd.read_csv(os.path.join(no_cna_dir,"tmp_CRUK0762_clustered_SNVs.tsv"),sep="\t")
df['implied F no CNA'] = df.apply(lambda row: row['var']/(0.5*row['ref']+row['var']), axis=1)
df = df[['anatomical_site_label', 'cluster_index', 'implied F no CNA']]
matrix_df = df.pivot(index='cluster_index', columns='anatomical_site_label', values='implied F no CNA')
matrix_df.to_csv(os.path.join(no_cna_dir,"CRUK0762_clustered_SNVs_no_cnas.tsv"), sep="\t")


AssertionError: cluster_index values do not go from 0 to max. Issue in: /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/tracerx_analysis/../../data/tracerx_nsclc/CRUK0762_no_cna_testing/CRUK0762_SNVs.tsv

In [ ]:
df